# Dependency

In [63]:
import pandas as pd
import numpy as np

# Data Preprocessing

In [5]:
fpath = 'C:\\Users\\hp\\Downloads\\yellow_cab_data\\yellow_cab_data.csv'
df = pd.read_csv(fpath)
df.tail(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
149997,2,2022-01-31 23:10:00,2022-01-31 23:20:00,NaN,2.02,NaN,NaN,170,50,0,10.40,0.0,0.5,1.46,0.00,0.3,15.16,NaN,NaN
149998,2,2022-01-31 23:53:00,2022-02-01 00:10:00,NaN,4.84,NaN,NaN,158,265,0,23.71,0.0,0.0,7.69,11.75,0.3,43.45,NaN,NaN
149999,2,2022-01-31 23:36:53,2022-01-31 23:42:51,NaN,1.32,NaN,NaN,90,170,0,8.00,0.0,0.5,2.39,0.00,0.3,13.69,NaN,NaN


In [11]:
# by EDA, we find that some entries are wrong (amount should always be non-negative)
df['fare_amount']=abs(df['fare_amount'])
df['extra']=abs(df['extra'])
df['mta_tax']=abs(df['mta_tax'])
df['tip_amount']=abs(df['tip_amount'])
df['tolls_amount']=abs(df['tolls_amount'])
df['improvement_surcharge']=abs(df['improvement_surcharge'])
df['total_amount']=abs(df['total_amount'])
df['congestion_surcharge']=abs(df['congestion_surcharge'])
df['airport_fee']=abs(df['airport_fee'])

In [12]:
df[df['airport_fee']<0].shape[0]

0

In [14]:
df = df.sort_values(by='tpep_pickup_datetime')
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
37509,2,2008-12-31 23:05:58,2009-01-01 19:57:57,4.0,7.4,1.0,N,138,80,1,22.0,0.00,0.5,4.81,0.00,0.3,28.86,0.0,1.25
16005,2,2009-01-01 00:46:11,2009-01-01 00:46:15,1.0,0.0,2.0,N,100,100,2,52.0,0.00,0.5,0.00,0.00,0.3,52.80,0.0,0.00
203,1,2022-01-01 00:00:23,2022-01-01 00:24:06,1.0,19.1,2.0,N,132,236,1,52.0,3.75,0.5,12.62,6.55,0.3,75.72,2.5,1.25


In [15]:
df = df[2:]
df.head(3)

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
203,1,2022-01-01 00:00:23,2022-01-01 00:24:06,1.0,19.10,2.0,N,132,236,1,52.0,3.75,0.5,12.62,6.55,0.3,75.72,2.5,1.25
145720,6,2022-01-01 00:01:45,2022-01-01 00:01:48,NaN,0.15,NaN,NaN,265,129,0,15.2,0.00,0.5,0.00,0.00,0.3,16.00,NaN,NaN
68,2,2022-01-01 00:02:45,2022-01-01 00:15:42,1.0,1.70,1.0,N,114,234,2,10.0,0.50,0.5,0.00,0.00,0.3,13.80,2.5,0.00


In [17]:
df = df[df['VendorID']<6]
df.shape[0]

149625

In [19]:
df[df['RatecodeID']==99].shape[0]

533

In [20]:
df = df[df['RatecodeID']!=99]
df.shape[0]

149092

In [21]:
#df.to_csv('data.csv')

# Database Management

In [25]:
#df.to_csv('data.csv')
import sqlite3
import sqlalchemy
#we provide cnt, sum, mean function for users
con = sqlite3.connect('taxi.db')

In [27]:
cursor = con.cursor()
cursor.execute('DROP TABLE IF EXISTS cab_Jan')
cursor.execute('CREATE TABLE taxi(VendorID,pickup_time,dropoff_time,passenger_cnt,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee)')
df.to_sql('cab_Jan',con=con,if_exists = 'replace')


In [48]:
query = """
SELECT sum(fare_amount)
from cab_Jan
where VendorID = 1 
and passenger_count = 1 
and trip_distance>-1
and trip_distance<2
and RatecodeID = 1
"""
pd.read_sql(query,con)

,sum(fare_amount)
0,133091.47


In [57]:
#API for users to query sum of fare amount
def query_fare_amt(vid,pcnt,dist_lb,dist_ub,rid):
    # if no constrain on the corresponding variable, user will be instructed to input -1
    query = "SELECT sum(fare_amount) from cab_Jan where trip_distance>-1"
    if(vid!=-1):
        query+=" and VendorID = "+str(vid)
    if(pcnt!=-1):
        query+=" and passenger_count = "+str(pcnt)
    if(dist_lb!=-1):
        query+=" and trip_distance >"+str(dist_lb)
    if(dist_ub!=-1):
        query+=" and trip_distance <"+str(dist_ub)
    if(rid != -1):
        query+=" and RatecodeID = "+str(rid)
    #print(query)
    return pd.read_sql(query,con)




In [58]:
#example of successfully calling API
query_fare_amt(1,1,-1,2,1)

,sum(fare_amount)
0,133091.47


In [67]:
query_fare_amt(-1,-1,-1,-1,-1)

,sum(fare_amount)
0,1905987.05


In [50]:
#example of not following the instruction(not using -1 u will get error)
query_fare_amt(-2,3)

TypeError: query_fare_amt() missing 3 required positional arguments: 'dist_lb', 'dist_ub', and 'rid'

In [51]:
#in the future we can create more API, like count, average

# Differential Privacy: Laplace Mechanism

In [53]:
def laplace_pdf(x, loc, scale):
    return (1/(2*scale)) * np.exp(-abs(x-loc)/scale)

In [70]:
res = query_fare_amt(-1,-1,-1,-1,-1).iloc[0,0]
drop_ind = np.random.choice(df.index, 1, replace = False)
df_temp = df.drop(drop_ind)
res1 = sum(df_temp['fare_amount'])
np.log(laplace_pdf(res,0,0.01*res)/laplace_pdf(res1,0,0.01*res1))

-4.459641503447932e-06

In [80]:
epsilon = []
for i in range(10000):
    drop_ind = np.random.choice(df.index, 1, replace = False)
    df_temp = df.drop(drop_ind)
    res1 = sum(df_temp['fare_amount'])
    epsilon.append(np.log(laplace_pdf(res,0,0.01*res)/laplace_pdf(res1,0,0.01*res1)))

In [81]:
max(epsilon)

0.0

In [82]:
min(epsilon)

-0.00013117423706416256

In [79]:
#so we choose epsilon = 0.0001